In [1]:
#add auto reload for src function testing
%load_ext autoreload
%autoreload 2

#let's add the project directory to our module path
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
    
#also import all of our modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from src import data_cleaning

#and here is our data directiory
data_folder = '../../data/'

# Objective

#### Match sales parcels to the parcel info data set

In [2]:
parcels = pd.read_csv(data_folder+'EXTR_Parcel_2019.csv', dtype={'PIN': 'string'})
sales = pd.read_csv(data_folder+'EXTR_RPSale_2019.csv', dtype={'PIN': 'string'})

I imported both of the filtered parcels, but I also want to find out if there were any missing parcels from either original csv. This would result in data loss.

In [3]:
sales_orig = pd.read_csv(data_folder+'EXTR_RPSale.csv', dtype={'Major': 'string', 'Minor': 'string'})
parcels_orig  = pd.read_csv(data_folder+'EXTR_Parcel.csv', dtype={'Major': 'string', 'Minor': 'string'}, encoding='latin-1')

### Cleaning the original 

I'm going to apply the same criteria to the full original data set and see how many data points there are.

#### Remove non-sales

In [4]:
only_sales = sales_orig[(sales_orig['SaleReason']==1)|(sales_orig['SaleReason']==18)]
only_sales

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,2,6,2,N,N,N,,1,8,
3,2254430,192304,9020,12/05/2006,0,20061207002200,,,,,...,3,11,15,N,N,N,N,18,2,18 45
4,685277,885730,0120,08/11/1982,0,198208170380,086,075,885730,P,...,3,2,15,N,N,N,,1,3,11
5,961971,165730,0360,08/25/1987,37500,198709010133,136,080,165730,P,...,1,0,2,N,N,N,N,1,7,
9,1709671,921440,0240,09/08/1999,292000,19990910001148,,,,,...,3,6,3,N,N,N,N,1,8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087938,1013951,247339,0160,07/22/1988,88000,198807291908,085,038,247339,P,...,3,0,2,N,N,N,N,1,8,
2087939,2845806,769791,0030,07/16/2016,133000,20170125000407,,,,,...,3,6,15,N,N,N,N,1,8,15 18 51
2087941,2201277,769681,0060,04/19/2006,225990,20060421002063,,,,,...,3,2,3,N,N,N,N,1,3,44
2087942,2971374,924600,0650,01/02/2019,0,,,,,,...,11,6,3,N,N,N,N,1,8,


In [5]:
sales_orig

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,2,6,2,N,N,N,,1,8,
1,1729614,172204,9157,12/21/1999,0,19991229001498,,,,,...,3,11,15,N,N,N,N,10,8,31 45
2,1729614,172204,9005,12/21/1999,0,19991229001498,,,,,...,3,11,15,N,N,N,N,10,8,31 45
3,2254430,192304,9020,12/05/2006,0,20061207002200,,,,,...,3,11,15,N,N,N,N,18,2,18 45
4,685277,885730,0120,08/11/1982,0,198208170380,086,075,885730,P,...,3,2,15,N,N,N,,1,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087939,2845806,769791,0030,07/16/2016,133000,20170125000407,,,,,...,3,6,15,N,N,N,N,1,8,15 18 51
2087940,1605944,409230,0885,04/16/1998,0,199804170621,005,034,409230,P,...,3,6,15,N,N,N,N,13,8,32 51
2087941,2201277,769681,0060,04/19/2006,225990,20060421002063,,,,,...,3,2,3,N,N,N,N,1,3,44
2087942,2971374,924600,0650,01/02/2019,0,,,,,,...,11,6,3,N,N,N,N,1,8,


#### Remove non-residential buildings

In [6]:
res_sales= only_sales[(only_sales['PropertyClass'] == 7)|(only_sales['PropertyClass'] == 8)]
res_sales

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,2,6,2,N,N,N,,1,8,
5,961971,165730,0360,08/25/1987,37500,198709010133,136,080,165730,P,...,1,0,2,N,N,N,N,1,7,
9,1709671,921440,0240,09/08/1999,292000,19990910001148,,,,,...,3,6,3,N,N,N,N,1,8,
10,2420985,234130,0265,12/04/2009,287000,20091209001633,,,,,...,3,6,3,N,N,N,N,1,8,
11,2043998,785996,0120,05/27/2004,5518000,20040602001705,,,,,...,1,6,3,N,N,N,N,1,7,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087937,1829636,125020,2965,07/10/2001,295000,20010716001201,,,,,...,3,6,3,N,N,N,N,1,8,45
2087938,1013951,247339,0160,07/22/1988,88000,198807291908,085,038,247339,P,...,3,0,2,N,N,N,N,1,8,
2087939,2845806,769791,0030,07/16/2016,133000,20170125000407,,,,,...,3,6,15,N,N,N,N,1,8,15 18 51
2087942,2971374,924600,0650,01/02/2019,0,,,,,,...,11,6,3,N,N,N,N,1,8,


#### Remove sales with a price of 0

In [7]:
no_zeros = res_sales[res_sales['SalePrice']!=0]
no_zeros

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,2,6,2,N,N,N,,1,8,
5,961971,165730,0360,08/25/1987,37500,198709010133,136,080,165730,P,...,1,0,2,N,N,N,N,1,7,
9,1709671,921440,0240,09/08/1999,292000,19990910001148,,,,,...,3,6,3,N,N,N,N,1,8,
10,2420985,234130,0265,12/04/2009,287000,20091209001633,,,,,...,3,6,3,N,N,N,N,1,8,
11,2043998,785996,0120,05/27/2004,5518000,20040602001705,,,,,...,1,6,3,N,N,N,N,1,7,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087936,1829636,125020,3010,07/10/2001,295000,20010716001201,,,,,...,3,6,3,N,N,N,N,1,8,45
2087937,1829636,125020,2965,07/10/2001,295000,20010716001201,,,,,...,3,6,3,N,N,N,N,1,8,45
2087938,1013951,247339,0160,07/22/1988,88000,198807291908,085,038,247339,P,...,3,0,2,N,N,N,N,1,8,
2087939,2845806,769791,0030,07/16/2016,133000,20170125000407,,,,,...,3,6,15,N,N,N,N,1,8,15 18 51


### Joining

Now I'm going to join this filtered data set with my parcel data to see if there is any missing info between the two data sets.

Make a pin column for each

In [8]:
qualifying_sales = data_cleaning.add_PIN_column(no_zeros)
parcels_orig = data_cleaning.add_PIN_column(parcels_orig)

/home/max/flatiron/chi-ds/module_2/week_3/projects/mod2_housing_sales/src/data_cleaning.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Major'] = df['Major'].apply(str)
/home/max/flatiron/chi-ds/module_2/week_3/projects/mod2_housing_sales/src/data_cleaning.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Minor'] = df['Minor'].apply(str)
/home/max/flatiron/chi-ds/module_2/week_3/projects/mod2_housing_sales/src/data_cleaning.py:48: SettingWithCopyWarning: 
A value is trying to be set on a cop

If I perform an inner join between qualify sales and parcels, it will return rows common only to both. If there are no parcels missing from the parcel list, then it will be the same size as the original qualifying sales data.

In [14]:
joined = pd.merge(qualifying_sales, parcels_orig, how='inner', on='PIN')

In [15]:
joined

,ExciseTaxNbr,Major_x,Minor_x,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,N,N,N,N,N,N,N,N,N,N
1,2692967,284150,0010,09/24/2014,390000,20140930000856,,,,,...,N,N,N,N,N,N,N,N,N,N
2,961971,165730,0360,08/25/1987,37500,198709010133,136,080,165730,P,...,N,N,N,N,N,N,N,N,N,N
3,1709671,921440,0240,09/08/1999,292000,19990910001148,,,,,...,N,N,N,N,N,N,N,N,N,N
4,1974120,921440,0240,07/15/2003,375000,20030721002311,,,,,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919298,2365030,213170,0770,09/24/2008,344950,20080926001536,,,,,...,N,N,N,N,N,N,N,N,N,N
919299,3028691,277110,4239,12/19/2019,955000,20200103000685,,,,,...,N,N,N,N,N,N,N,N,N,N
919300,2667830,785641,0030,05/05/2014,1030000,20140515000282,,,,,...,N,N,N,N,N,N,N,N,N,N
919301,2942886,886030,0550,07/16/2016,900000,20180720000629,,,,,...,N,N,N,N,N,N,N,N,N,N


In [16]:
qualifying_sales

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,PIN
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,6,2,N,N,N,,1,8,,2841500010
5,961971,165730,0360,08/25/1987,37500,198709010133,136,080,165730,P,...,0,2,N,N,N,N,1,7,,1657300360
9,1709671,921440,0240,09/08/1999,292000,19990910001148,,,,,...,6,3,N,N,N,N,1,8,,9214400240
10,2420985,234130,0265,12/04/2009,287000,20091209001633,,,,,...,6,3,N,N,N,N,1,8,,2341300265
11,2043998,785996,0120,05/27/2004,5518000,20040602001705,,,,,...,6,3,N,N,N,N,1,7,45,7859960120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087936,1829636,125020,3010,07/10/2001,295000,20010716001201,,,,,...,6,3,N,N,N,N,1,8,45,1250203010
2087937,1829636,125020,2965,07/10/2001,295000,20010716001201,,,,,...,6,3,N,N,N,N,1,8,45,1250202965
2087938,1013951,247339,0160,07/22/1988,88000,198807291908,085,038,247339,P,...,0,2,N,N,N,N,1,8,,2473390160
2087939,2845806,769791,0030,07/16/2016,133000,20170125000407,,,,,...,6,15,N,N,N,N,1,8,15 18 51,7697910030


In [30]:
len(qualifying_sales) - len(joined)

12627

It is slightly smaller. Let's check the value_counts( for PIN and see if there are still the same number of sales for each house.

In [17]:
qualifying_sales.PIN.value_counts()

0524079007    27
0646900010    20
7169200242    20
0646900030    18
0114100979    17
              ..
7203100350     1
5095020100     1
0859000030     1
2595300400     1
0226069035     1
Name: PIN, Length: 459479, dtype: int64

In [18]:
joined.PIN.value_counts()

0524079007    27
0114100979    17
0723039070    13
2225300144    11
3622069077    11
              ..
1442800010     1
2787720750     1
2726049188     1
8564600210     1
0226069035     1
Name: PIN, Length: 450792, dtype: int64

It looks like all the sales entries for 0646900010 were dropped. Let's make sure they aren't in the original parcel data.

In [19]:
parcels_orig[parcels_orig['PIN'] == '0646900010']

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,PIN


## Encoding

And it's definitely not in there.
I wonder if these values change when the encoding changes.

We'll test out latin-8 just to see.

In [20]:
parcels_latin_8 = pd.read_csv(data_folder+'EXTR_Parcel.csv', dtype={'Major': 'string', 'Minor': 'string'}, encoding='iso8859_14')

In [21]:
parcels_latin_8[parcels_orig['PIN'] == '0646900010']

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems


In [22]:
parcels_latin_8 = data_cleaning.add_PIN_column(parcels_latin_8)

In [26]:
parcels_latin_8[parcels_latin_8['PIN'] == '0646900010']

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,PIN


If I do an inner join on parcels_latin_8 and parcels_orig and the number of entries remains the same, then the encoding probably does not affect the creation of PINs.

In [27]:
pd.merge(parcels_latin_8.PIN, parcels_orig.PIN, on='PIN', how='inner')

,PIN
0,8078410410
1,2019200030
2,7550800015
3,8886000135
4,6430000185
...,...
614822,8819900840
614823,6021500990
614824,1624059314
614825,7812800885


In [28]:
parcels_orig

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,PIN
0,807841,0410,,SUMMER RIDGE DIV NO. 02,41,,6,25,22,SW,...,N,N,N,N,N,N,N,N,N,8078410410
1,201920,0030,,DEVILLE MANOR,3,,4,21,28,SE,...,N,N,N,N,N,N,N,N,N,2019200030
2,755080,0015,,SANDER'S TO GILMAN PK & SALMON BAY,3,1,3,25,11,NW,...,N,N,N,N,N,N,N,N,N,7550800015
3,888600,0135,,VASHON GARDENS ADD,21,,3,22,8,NE,...,N,N,N,N,N,N,N,N,N,8886000135
4,643000,0185,,OSNERS 1ST ADD,37-38,1,4,25,6,NW,...,N,N,N,N,N,N,N,N,N,6430000185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614822,881990,0840,,UNIVERSITY HOME TRS,4 & 13,8,4,25,10,NW,...,N,N,N,N,N,N,N,N,N,8819900840
614823,602150,0990,,NELSON H E ADD,7,5,3,25,2,NE,...,N,N,N,N,N,N,N,N,N,6021500990
614824,162405,9314,,NaN,,,5,24,16,NW,...,N,N,N,N,N,N,N,N,N,1624059314
614825,781280,0885,,SKYWAY PARK ADD,11,9,4,23,12,SW,...,N,N,N,N,N,N,N,N,N,7812800885


In [29]:
parcels_latin_8

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,PIN
0,807841,0410,,SUMMER RIDGE DIV NO. 02,41,,6,25,22,SW,...,N,N,N,N,N,N,N,N,N,8078410410
1,201920,0030,,DEVILLE MANOR,3,,4,21,28,SE,...,N,N,N,N,N,N,N,N,N,2019200030
2,755080,0015,,SANDER'S TO GILMAN PK & SALMON BAY,3,1,3,25,11,NW,...,N,N,N,N,N,N,N,N,N,7550800015
3,888600,0135,,VASHON GARDENS ADD,21,,3,22,8,NE,...,N,N,N,N,N,N,N,N,N,8886000135
4,643000,0185,,OSNERS 1ST ADD,37-38,1,4,25,6,NW,...,N,N,N,N,N,N,N,N,N,6430000185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614822,881990,0840,,UNIVERSITY HOME TRS,4 & 13,8,4,25,10,NW,...,N,N,N,N,N,N,N,N,N,8819900840
614823,602150,0990,,NELSON H E ADD,7,5,3,25,2,NE,...,N,N,N,N,N,N,N,N,N,6021500990
614824,162405,9314,,NaN,,,5,24,16,NW,...,N,N,N,N,N,N,N,N,N,1624059314
614825,781280,0885,,SKYWAY PARK ADD,11,9,4,23,12,SW,...,N,N,N,N,N,N,N,N,N,7812800885
